# Ch-11. 그룹 연산
## 11-1. 데이터 집계

### - (p.218) groupby 메서드

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os

In [2]:
path = os.getcwd() + '\data'
os.chdir(path)
os.getcwd()

'C:\\Users\\James\\Documents\\GitHub\\doit_pandas\\data'

In [4]:
gapminder_raw = pd.read_csv('gapminder.tsv', sep = '\t')
df = gapminder_raw.copy()
df.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106


In [5]:
df.groupby('year')['lifeExp'].mean()

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

### - (p.219) 분할-반영-결합(Split-Apply-Combine)

#### (1) 분할

In [6]:
years = df['year'].unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

#### (2) 반영

In [12]:
y1952 = df.loc[df['year'] == 1952, :]
y1952.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
12,Albania,Europe,1952,55.230,1282697,1601.056136
24,Algeria,Africa,1952,43.077,9279525,2449.008185
36,Angola,Africa,1952,30.015,4232095,3520.610273
48,Argentina,Americas,1952,62.485,17876956,5911.315053


In [13]:
y1952_mean = y1952['lifeExp'].mean()
y1952_mean

49.05761971830987

In [14]:
y1957 = df.loc[df['year'] == 1957, :]
y1957_mean = y1957['lifeExp'].mean()
y1957_mean

51.507401126760534

In [16]:
# years 값 모두 반복

#### (3) 결합

In [18]:
df2 = pd.DataFrame({'year' : [years[0], years[1]],
                    'mean' : [y1952_mean, y1957_mean]})

In [19]:
df2

,year,mean
0,1952,49.057620
1,1957,51.507401


### - (p.221) 집계 메서드

#### * count : 누락값 제외한 데이터 갯수

#### * size : 누락값 포함한 데이터 갯수

#### * mean, sum, median, min, max, std, var

#### * quantile(q=0.25) : 백분위수 25%

#### * quantile(q=0.50) : 백분위수 50%

#### * quantile(q=0.75) : 백분위수 75%

#### * sem : 평균의 표준편차

#### * describe

#### * first : 첫 번째 행

#### * last : 마지막 행

#### * nth : n번째 행 

### - (p.221) groupby + agg + 직접 만든 함수

In [21]:
# 평균 구하는 함수 직접 만들기
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    
    return sum / n

In [24]:
df.groupby('year')['lifeExp'] \
  .agg(my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

#### * 2개의 인자값을 받는 사용자 함수

In [25]:
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [26]:
# 연도별 평균 수명에서 전체 평균 수명을 뺀 값 구하기

In [27]:
global_mean = df['lifeExp'].mean()
global_mean

59.47443936619714

In [28]:
df.groupby('year')['lifeExp'] \
  .agg(my_mean_diff, diff_value=global_mean)

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

### - (p.224) 여러 개의 집계 메서드 한 번에 사용

#### * 리스트로 전달

In [39]:
# lifeExp 열의 0이 아닌 값의 갯수, 평균, 표준편차
df.groupby('year')['lifeExp'] \
  .agg([np.count_nonzero, np.mean, np.std]) \
  .head()

,count_nonzero,mean,std
year,,,
1952,142,49.057620,12.225956
1957,142,51.507401,12.231286
1962,142,53.609249,12.097245
1967,142,55.678290,11.718858
1972,142,57.647386,11.381953


#### * 딕셔너리로 전달

In [40]:
df.groupby('year') \
  .agg({'lifeExp' : 'mean', 
        'pop' : 'median',
        'gdpPercap' : 'median'}) \
  .head()

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334740
1972,57.647386,5877996.5,3339.129407


#### * 이렇게 해라! 가장 확실한 방법

In [38]:
df.groupby('year') \
  .agg(mean_lifeExp = ('lifeExp', 'mean'),
       median_pop = ('pop', 'median'),
       median_gPc = ('gdpPercap', 'median')) \
  .head()

,mean_lifeExp,median_pop,median_gPc
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334740
1972,57.647386,5877996.5,3339.129407


## 11-2. 데이터 변환

### - (p.226) 표준점수 계산

In [51]:
# 표준점수(Z점수, z-score) = 데이터의 평균관 표준편차의 차이
# 표준화 >> 평균값 = 0, 표준편차 = 1

In [44]:
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [45]:
transform_z = df.groupby('year')['lifeExp'].transform(my_zscore)
transform_z.head()

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64

In [52]:
df.shape

(1704, 6)

In [53]:
transform_z.shape

(1704,)

### - (p.227) 누락값을 평균값으로 처리

In [54]:
import seaborn as sns

In [55]:
np.random.seed(42)

In [56]:
tips_10 = sns.load_dataset('tips').sample(10)

In [57]:
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [59]:
# 성별 데이터 갯수 확인
tips_10.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


In [62]:
# 성별을 구분하여 total_bill 열의 평균값 구하는 함수
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [64]:
tips_10['fill_total_bill'] = tips_10.groupby('sex')['total_bill'].transform(fill_na_mean)

In [65]:
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
24,19.82,3.18,Male,No,Sat,Dinner,2,19.8200
6,8.77,2.00,Male,No,Sun,Dinner,2,8.7700
153,NaN,2.00,Male,No,Sun,Dinner,4,17.9525
211,NaN,5.16,Male,Yes,Sat,Dinner,4,17.9525
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.9300
176,NaN,2.00,Male,Yes,Sun,Dinner,2,17.9525
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.4400
124,12.48,2.52,Female,No,Thur,Lunch,2,12.4800
9,14.78,3.23,Male,No,Sun,Dinner,2,14.7800
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.3800


## 11-3. 데이터 필터링